In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
df = pd.read_csv("../../data_files/Iris.csv")

In [3]:
df['Species'].describe()

count             150
unique              3
top       Iris-setosa
freq               50
Name: Species, dtype: object

In [4]:
species=df['Species'].unique()
len(species)

3

In [5]:
mapping = {}
for i, species in enumerate(df['Species'].unique()):
    mapping[species] = i

df['m_Species'] = df['Species'].map(mapping)
type(df['m_Species'].unique())


numpy.ndarray

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
 6   m_Species      150 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 8.3+ KB


In [7]:
# define y
y = df['m_Species'].values

# define X (features)
X = df.drop(['Species', 'm_Species', 'Id'], axis=1, errors='ignore').values

# check
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (150, 4)
y shape: (150,)


`.values` converts the Pandas **DataFrame** (rows and columns) into **Numpy Array** -> just numbers, which is exactly what PyTorch wants.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()       # scaler object
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# conver to pytorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)

X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

In [12]:
# move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

print("Tensor Shape: ", X_train_tensor.shape)

Using device: cuda
Tensor Shape:  torch.Size([120, 4])


In [17]:
#creating the model
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        # Layer 1: Input (4) -> Hidden (16)
        self.fc1 = nn.Linear(4, 16)
        
        # Activation Function: ReLU (Rectified Linear Unit)
        # It turns negative numbers to 0. It introduces "non-linearity".
        self.relu = nn.ReLU()
        
        # Layer 2: Hidden (16) -> Output (3)
        self.fc2 = nn.Linear(16, 3)
    
    def forward(self, x):
        # x is the input data
        x = self.fc1(x)  # Pass through Layer 1
        x = self.relu(x) # Apply activation logic
        x = self.fc2(x)  # Pass through Layer 2
        return x

# Initialize the model and move it to the GPU
model = SimpleMLP().to(device)
print(model)

SimpleMLP(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)


In [ ]:
# 1. The Loss Function
# CrossEntropyLoss is the standard for Classification (picking 1 out of 3 flowers).
criterion = nn.CrossEntropyLoss()       # loss function = how wrong your model is

# 2. The Optimizer
# Adam is a smart algorithm that adjusts the learning rate automatically.
# lr=0.01 is the "Learning Rate" (how big of a step to take).
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [15]:
epochs = 100
loss_history = []

print("--- Starting Training ---")

for epoch in range(epochs):
    # STEP 1: Clear the old gradients (so they don't pile up)
    optimizer.zero_grad()
    
    # STEP 2: Forward Pass (The model guesses)
    outputs = model(X_train_tensor)
    
    # STEP 3: Calculate Loss (How wrong was the guess?)
    loss = criterion(outputs, y_train_tensor)
    
    # STEP 4: Backward Pass (Calculate corrections/gradients)
    loss.backward()
    
    # STEP 5: Update Weights (The optimizer fixes the model)
    optimizer.step()
    
    # Track progress
    loss_history.append(loss.item())
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("--- Training Complete ---")

--- Starting Training ---
Epoch [10/100], Loss: 0.7732
Epoch [20/100], Loss: 0.4854
Epoch [30/100], Loss: 0.3631
Epoch [40/100], Loss: 0.3022
Epoch [50/100], Loss: 0.2516
Epoch [60/100], Loss: 0.2035
Epoch [70/100], Loss: 0.1593
Epoch [80/100], Loss: 0.1258
Epoch [90/100], Loss: 0.1041
Epoch [100/100], Loss: 0.0892
--- Training Complete ---


In [16]:
# Turn off gradient calculation (saves memory/speed for testing)
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    
    # The model gives probabilities for [Class 0, Class 1, Class 2].
    # We take the index of the highest score using torch.max
    _, predicted = torch.max(test_outputs, 1)
    
    # Calculate accuracy
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / len(y_test_tensor)

print(f"✅ Accuracy on Test Set: {accuracy * 100:.2f}%")

✅ Accuracy on Test Set: 100.00%
